In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string
from collections import Counter
import spacy
import en_core_web_sm
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import cohen_kappa_score, make_scorer
from xgboost import XGBClassifier
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#importing nlp library
import nltk
import re
nlp = en_core_web_sm.load()
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')

test_label = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip')
sample_submission = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')


In [ ]:
sample_submission.head()

In [ ]:
#test_label.head

In [ ]:
train.head()

In [ ]:
test.head()

## **Pré-processamento**

In [ ]:
def pre_processamento(all_words):
    en_stops = set(stopwords.words('english'))
    return ' '.join(token.lower() for token in word_tokenize(all_words)
                        if token.isalpha() and token not in en_stops)

In [ ]:
#stopwords inglês
#en_stops

In [ ]:
train['text_no_ponctuation_number'] = train['comment_text'].apply(lambda x: [token for token in str(x) if token not in string.punctuation and not token.isnumeric()])
train['text_no_ponctuation_number'] = train['text_no_ponctuation_number'].apply(lambda x: ''.join(x))

test['text_no_ponctuation_number'] = test['comment_text'].apply(lambda x: [token for token in str(x) if token not in string.punctuation and not token.isnumeric()])
test['text_no_ponctuation_number'] = test['text_no_ponctuation_number'].apply(lambda x: ''.join(x))

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
vectorizer = TfidfVectorizer(use_idf=True, ngram_range=(1,2))

X = vectorizer.fit_transform(train.text_no_ponctuation_number.apply(pre_processamento))
vc_test = vectorizer.transform(test.text_no_ponctuation_number.apply(pre_processamento))

In [ ]:
#Classes
Y = train[['toxic','severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

#Divisão Teste e Treino
x_train, x_test, class_train, class_test = train_test_split(X, Y, test_size=0.25)

In [ ]:
models = [
          ('MNB', MultinomialNB())
        ]

for name, model in models:
        model = OneVsRestClassifier(model)
        model.fit(x_train, class_train)
        preds = model.predict(x_test)
        print(name)
        print(metrics.classification_report(preds, class_test))

In [ ]:
y_preds = model.predict_proba(vc_test)

In [ ]:
sample_submission = pd.concat([test.loc[:, ('id',)], pd.DataFrame(data=y_preds, columns=sample_submission.columns[1:])], axis=1)

In [ ]:
print(sample_submission)
sample_submission.to_csv('./submission.csv', index=False)